# Fourier Neural Operators (FNO)

In this notebook we will introduce the brief theory behind the Fourier Neural Operators and use them to solve a data-driven Darcy flow problem. The example is adopted from the [paper](https://arxiv.org/pdf/2010.08895.pdf) by Zongyi Li et al. You can also refer to the FNO example and theory from [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/user_guide/neural_operators/darcy_fno.html) for additional details. 

### Learning Outcomes
1. How to use Modulus for setting up data-driven problem using Fourier Neural Operator (FNO)
    1. How to load grid data and setup data-driven constraints
    2. How to use eager and lazy data loading

## FNO Theory

Fourier neural operator (FNO) is a data-driven architecture which can be used to parameterize solutions for a distribution of PDE solutions. The key feature of FNO is the spectral convolutions: operations that place the integral kernel in Fourier space. The spectral convolution (Fourier integral oprator) is defined as follows:

\begin{equation}
(\mathcal{K}(\mathbf{w})\phi)(x) = \mathcal{F}^{-1}(R_{\mathbf{W}}\cdot \left(\mathcal{F}\right)\phi)(x), \quad \forall x \in D
\end{equation}

where $\mathcal{F}$ and $\mathcal{F}^{-1}$ are the forward and inverse Fourier transforms, respectively.
$R_{\mathbf{w}}$ is the transformation which contains the learnable parameters $\mathbf{w}$. Note this operator is calculated
over the entire *structured Euclidean* domain $D$ discretized with $n$ points.

Fast Fourier Transform (FFT) is used to perform the Fourier transforms efficiently and the resulting transformation $R_{\mathbf{w}}$ is just finite size matrix of learnable weights. In side the spectral convolution, the Fourier coefficients are truncated to only the lower modes which intern allows explicit control over the dimensionality of the spectral space and linear operator.

The FNO model is a the composition of a fully-connected "lifting" layer, $L$ spectral convolutions with point-wise linear skip connections and a decoding point-wise fully-connected neural network at the end.

\begin{equation}
u_{net}(\Phi;\theta) = \mathcal{Q}\circ \sigma(W_{L} + \mathcal{K}_{L}) \circ ... \circ \sigma(W_{1} + \mathcal{K}_{1})\circ \mathcal{P}(\Phi), \quad \Phi=\left\{\phi(x); \forall x \in D\right\}
\end{equation}

in which $\sigma(W_{i} + \mathcal{K}_{i})$ is the spectral convolution layer $i$ with the point-wise linear transform $W_{i}$ and activation function $\sigma(\cdot)$. $\mathcal{P}$ is the point-wise lifting network that projects the input into a higher-dimensional latent space, $\mathcal{P}: \mathbb{R}^{d_in} \rightarrow \mathbb{R}^{k}$.

Similarly $\mathcal{Q}$ is the point-wise fully-connected decoding network, $\mathcal{P}: \mathbb{R}^{k} \rightarrow \mathbb{R}^{d_out}$. Since all fully-connected components of FNO are point-wise operations, the model is invariant to the dimensionality of the input.


**Note:** While FNO is technically invariant to the dimensionality of the discretized domain $D$, this domain *must* be a structured grid in Euclidean space. The inputs to FNO are analgous to images, but the model is invariant to the image resolution.


## Problem Description (Darcy Flow)

We will demonstrate the use of Fourier Neural Operators on a 2D Darcy flow problem. The Darcy PDE is a second order, elliptic PDE with the following form:

\begin{equation}
-\nabla \cdot \left(k(\textbf{x})\nabla u(\textbf{x})\right) = f(\textbf{x}), \quad \textbf{x} \in D,
\end{equation}

in which $u(\textbf{x})$ is the flow pressure, $k(\textbf{x})$ is the permeability field and $f(\cdot)$ is the
forcing function. The Darcy flow can parameterize a variety of systems including flow through porous media, elastic materials 
and heat conduction. Here you will define the domain as a 2D unit square  $D=\left\{x,y \in (0,1)\right\}$ with the boundary condition $u(\textbf{x})=0, \textbf{x}\in\partial D$. Recall that FNO requires a structured Euclidean input such that $D = \textbf{x}_{i}$ where $i \in \mathbb{N}_{N\times N}$. Thus both the permeability and flow fields are discretized into a 2D matrix $\textbf{K}, \textbf{U} \in \mathbb{R}^{N \times N}$.

This problem develops a surrogate model that learns the mapping between a permeability field and the pressure field,
$\textbf{K} \rightarrow \textbf{U}$, for a distribution of permeability fields $\textbf{K} \sim p(\textbf{K})$.
This is a key distinction of this problem from other PINN examples, you are *not* learning just a single solution but rather a distribution.

<img src="fno_darcy.png" alt="Drawing" style="width: 900px;"/>

## Case Setup

The example covered in this notebook is a data-driven problem. This means that before starting any coding we need to make sure that we have both the training and the validation data. The training and validation datasets for this example can be found on the [Fourier Neural Operator Github page](https://github.com/zongyi-li/fourier_neural_operator). The script [`utilities.py`](../../source_code/darcy/utilities.py) is an automated script for downloading and converting this dataset. This requires the package [gdown](https://github.com/wkentaro/gdown) which can be easily installed through `pip install gdown`. 

**Note:** In this notebook we will walk through the contents of the [`darcy_FNO_lazy.py`](../../source_code/darcy/darcy_FNO_lazy.py) script.

In [ ]:
!pip install gdown

## Configuration

Before we dive into the contents of the script, lets have a quick look at the configuration for this problem. The configuration for this problem is fairly standard within Modulus. A specific FNO architecture is defined for this example inside of the config file. These settings were derived through an automated hyper-parameter sweep using Hydra multirun. The most important paramater for the FNO models is the `dimension` which tells Modulus to load a 1D, 2D or 3D FNO architecture. `nr_fno_layers` are the number of Fourier convolution layers in the model and `fno_layer_size` are the size of the latent embedded features inside the model. The contents of the [`config_FNO.yaml`](../../source_code/darcy/conf/config_FNO.yaml) are shown below. 

```yaml
defaults :
  - modulus_default
  - /arch/conv_fully_connected_cfg@arch.decoder
  - /arch/fno_cfg@arch.fno
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

arch:
  decoder:
    input_keys: [z, 32]
    output_keys: sol
    nr_layers: 1
    layer_size: 32

  fno:
    input_keys: coeff
    dimension: 2
    nr_fno_layers: 4
    fno_modes: 12
    padding: 9

scheduler:
  decay_rate: 0.95
  decay_steps: 1000

training:
  rec_results_freq : 1000
  max_steps : 10000

batch_size:
  grid: 32
  validation: 32
```

Now let's start with importing the required packages and modules

```python
import modulus
from modulus.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.key import Key

from modulus.solver import Solver
from modulus.domain import Domain
from modulus.domain.constraint import SupervisedGridConstraint
from modulus.domain.validator import GridValidator
from modulus.dataset import HDF5GridDataset

from modulus.utils.io.plotter import GridValidatorPlotter

from utilities import download_FNO_dataset
```

## Loading the Data

For this data-driven problem, the first step is to get the training data into Modulus. Prior to loading data, we can set any normalization value that we want to apply to the data. For this dataset, we calucalted the scale and shift parameters for both the input permeability field and output pressure. Then, we set this normalization inside Modulus by providing the scale/shift to each key, `Key(name, scale=(shift, scale))`. 

```python
@modulus.main(config_path="conf", config_name="config_FNO")
def run(cfg: ModulusConfig) -> None:

    # load training/ test data
    input_keys = [Key("coeff", scale=(7.48360e00, 4.49996e00))]
    output_keys = [Key("sol", scale=(5.74634e-03, 3.88433e-03))]
```

There are two approaches for loading data: First, use eager loading where you immediately read the entire datast into memory at one time. Alternatively, you can use lazy loading where the data is loaded on a per example basis as the model needs it for training. The eager loading eliminates potential overhead from reading data from disc during training, however this cannot scale to large datasets. Lazy loading is used in this example for the training dataset to demonstrate this utility for larger problems. This data is in HDF5 format which is ideal for lazy loading using the *HDF5DataFile* object.

```python
    download_FNO_dataset("Darcy_241", outdir="datasets/")
    train_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5")
    test_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5")

    # make datasets
    train_dataset = HDF5GridDataset(train_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=1000)
    test_dataset = HDF5GridDataset(test_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=100)
```

**Note:** The key difference when setting up eager versus lazy loading is the object passed in the variable dictionaries *invar_train* and *outvar_train*. In eager loading these dictionaries should be of the type `Dict[str: np.array]`, where each variable is a numpy array of data. Lazy loading uses dictionaries of the type `Dict[str: DataFile]`, consisting of `DataFile` objects which are classes that are used to map between example index and the datafile.  

## Initializing the Model

Initializing the model and domain follows the same steps as the other PINN models we saw earlier. 

```python
    # make list of nodes to unroll graph on
    decoder_net = instantiate_arch(
        cfg=cfg.arch.decoder,
        output_keys=output_keys,
    )
    fno = instantiate_arch(
        cfg=cfg.arch.fno,
        input_keys=input_keys,
        decoder_net=decoder_net,
    )
    nodes = [fno.make_node('fno')]

    # make domain
    domain = Domain()
```

## Adding the Data Constraints

For the physics-informed problems in Modulus, we typically need to define a geometry and constraints based on boundary conditions and governing equations. Here, the only constraint is a `SupervisedGridConstraint` which performs standard supervised training on grid data. This constraint supports the use of multiple workers, which are particularly important when using lazy loading. 

```python
    # add constraints to domain
    supervised = SupervisedGridConstraint(
        nodes=nodes,
        dataset=train_dataset,
        batch_size=cfg.batch_size.grid,
        num_workers=4,  # number of parallel data loaders
    )
    domain.add_constraint(supervised, "supervised")
```

**Note:** Grid data refers to data that can be defined in a tensor like an image. Inside Modulus, this grid of data typically represents a spatial domain and should follow the standard dimensionality of `[batch, channel, xdim, ydim, zdim]` where channel is the dimensionality of your state variables. Both Fourier and convolutional models use grid-based data to efficiently learn and predict entire domains in one forward pass, which contrasts to the pointwise predictions of standard PINN approaches. 

## Adding Data Validator

The validation data is then added to the domain using `GridValidator` which should be used when dealing with structured data. 

```python
    # add validator
    val = GridValidator(
        nodes,
        dataset=test_dataset,
        batch_size=cfg.batch_size.validation,
        plotter=GridValidatorPlotter(n_examples=5),
    )
    domain.add_validator(val, "test")
```

## Solver and Training the Model 

We can create a solver by using the domain we just created along with the other configurations that define the optimizer choices, settings using Modulus’ `Solver` class. The solver can then be executed using the solve method

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

The training for the problem can be simply started by executing the python script similar to the examples we saw earlier. 

In [ ]:
import os
os.environ["RANK"]="0"
os.environ["WORLD_SIZE"]="1"
os.environ["MASTER_ADDR"]="localhost"

In [ ]:
!python ../../source_code/darcy/darcy_FNO_lazy.py

## Results and Post-processing

The checkpoint directory is saved based on the results recording frequency specified in the `rec_results_freq` parameter of its derivatives. The network directory folder contains several plots of different validation predictions. Several are shown below, and you can see that the model is able to accurately predict the pressure field for permeability fields it had not seen previously. 

FNO validation predictions. (Left to right) Input permeability, true pressure, predicted pressure, error. 

<img src="fno_darcy_pred1.png" alt="Drawing" style="width: 900px;"/>
<img src="fno_darcy_pred2.png" alt="Drawing" style="width: 900px;"/>
<img src="fno_darcy_pred3.png" alt="Drawing" style="width: 900px;"/>


# Licensing
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).
